In [1]:
import os
import sys
import pickle
import numpy as np
import random
sys.path.insert(0, os.getcwd())

os.chdir('..')
from lib.utils.tools import read_pkl
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_aihub import DataReaderAIHUB
from tqdm import tqdm

In [2]:
#datareader = DataReaderH36M(n_frames=243, sample_stride=1, data_stride_train=81, data_stride_test=243, dt_file = 'h36m_sh_conf_cam_source_final.pkl', dt_root='data/motion3d/')
datareader = DataReaderAIHUB(n_frames=243, sample_stride=1, data_stride_train=81, data_stride_test=243, dt_file = 'aihub_30.pkl', dt_root='data/motion3d/')


In [3]:
train_data, test_data = datareader.read_2d()

In [4]:
train_labels, test_labels = datareader.read_3d()

In [5]:
datareader.sample_stride

1

In [6]:
vid_list_train = datareader.dt_dataset['train']['source'][::datareader.sample_stride]
vid_list_test = datareader.dt_dataset['test']['source'][::datareader.sample_stride]  

In [7]:
len(vid_list_train)

17322

In [18]:
def split_clips(vid_list, n_frames, data_stride):
    result = []
    n_clips = 0
    st = 0
    i = 0
    saved = set()
    while i<len(vid_list):
        i += 1
        #print(i)
        if i-st == n_frames: # 시작 지점부터 n_frames마다
            #print(i, st, n_frames)
            result.append(range(st,i))
            saved.add(vid_list[i-1])
            st = st + data_stride
            n_clips += 1
        if i==len(vid_list):
            break
        if vid_list[i]!=vid_list[i-1]: 
            #print(vid_list[i], vid_list[i-1])
            if not (vid_list[i-1] in saved):
                resampled = resample(i-st, n_frames) + st
                result.append(resampled)
                saved.add(vid_list[i-1])
            st = i
        #print(result)
    return result

def resample(ori_len, target_len, replay=False, randomness=True):
    if replay:
        if ori_len > target_len:
            st = np.random.randint(ori_len-target_len)
            return range(st, st+target_len)  # Random clipping from sequence
        else:
            return np.array(range(target_len)) % ori_len  # Replay padding
    else:
        if randomness:
            even = np.linspace(0, ori_len, num=target_len, endpoint=False)
            if ori_len < target_len:
                low = np.floor(even)
                high = np.ceil(even)
                sel = np.random.randint(2, size=even.shape)
                result = np.sort(sel*low+(1-sel)*high)
            else:
                interval = even[1] - even[0]
                result = np.random.random(even.shape)*interval + even
            result = np.clip(result, a_min=0, a_max=ori_len-1).astype(np.uint32)
        else:
            result = np.linspace(0, ori_len, num=target_len, endpoint=False, dtype=int)
        return result

In [19]:
datareader.n_frames

243

In [20]:
vid_list_test[:500]

array(['res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3', 'res_30_M170B_3', 'res_30_M170B_3',
       'res_30_M170B_3',

In [21]:
split_id_test = split_clips(vid_list_test, datareader.n_frames, data_stride=datareader.data_stride_test)

In [23]:
len(split_id_test)

55